In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_columns = 999

In [2]:
# Cargar datos

import glob
path =r'./microdatos' # use your path
allFiles = glob.glob(path + "/usu_individual*.txt")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, delimiter = ';')
    list_.append(df)
datos = pd.concat(list_)

usd_ars = pd.read_csv('./usd_ars.csv')
usd_ars = usd_ars.groupby(['ANO4', 'TRIMESTRE'])[['USD/ARS']].mean().reset_index()
datos = datos.merge(usd_ars)
datos['ingreso_usd'] = datos['P47T'] / datos['USD/ARS']

/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (102,105,134,136,143,145) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (102,105,134,136,137,142,143,144,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (102,105,134,137,143,146) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (102,105,134,136,137,143,145,146)

## Cargar radios censales

In [15]:
# RADIO 62,030,301 is buggy. Big patch in south Bs As
import urllib
import json
import shapely.geometry as geometry
import pyproj

p1 = pyproj.Proj(init='epsg:3857') # 'epsg:3395' or 'epsg:3857'
p2 = pyproj.Proj(proj='latlong',datum='WGS84')
df_list = []
data_list = []

# Breakup for country-wide query
for start, end in [(0, 25000), (25000, 50000), (50000, 75000)]:

    #total pais
    url = "https://services1.arcgis.com/gKFHG0iKXPnKkg1l/arcgis/rest/services/Radios_censales/FeatureServer/0/query?"\
    +"where=OBJECTID+>=+"+str(start)+"+AND+OBJECTID+<"+str(end)+"&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnDistinctValues=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

    res = urllib.urlopen(url)
    data = json.loads(res.read())

    # Format for Geojson
    for d in data["features"]:
        d["geometry"]["coordinates"] = d["geometry"].pop("rings")
        # Project
        d["geometry"]["coordinates"][0] = [list(pyproj.transform(p1, p2, point[0], point[1])) for point in d["geometry"]["coordinates"][0]]
        
        # set rounding level according to polygon size
        bounds = geometry.Polygon(d["geometry"]["coordinates"][0]).bounds
        round_lev =  - int(min(np.floor(np.log10(bounds[2] - bounds [0]) - 1.5), 
                        np.floor(np.log10(bounds[3] - bounds [1]) - 1.5)))
        d["geometry"]["coordinates"][0] = np.round(d["geometry"]["coordinates"][0], round_lev).tolist()
        
        d["geometry"]["type"] = "Polygon"

        d["properties"] = d.pop("attributes")

        d["type"] = "Feature"
        
    data_list += [data]
    
    df = pd.DataFrame([e['properties'] for e in data['features']]) 
    df_list += [df]
    
df = pd.concat(df_list).reset_index(drop = True)

data = data_list[0]
if len(data_list) > 1:
    for di in data_list[1:]: 
        for k in di:
            try:
                data[k].update(di[k])
            except AttributeError: # in case it is a list
                data[k] += di[k]

### Cargar respuestas al censo

In [21]:
import unidecode

pregs_dict = {'names': ['PERSONA-CONDACT', 'PERSONA-EDADAGRU','PERSONA-P02', 'PERSONA-P05', 
                        'PERSONA-P07', 'PERSONA-P08', 'PERSONA-P09', 'PERSONA-P10',
                       'VIVIENDA-URP', 'VIVIENDA-V01'],
              'prefix': ['Actividad ', 'Grupo etario', 'Sexo ', 'Nacionalidad ', 
                         'Sabe leer ', 'Asist escuela ', 'Nivel educativo ', 'Terminado ', 
                        'Urbana o rural ', 'Tipo de vivienda particular ',
                        ]}
df_list = []
for i in range(len(pregs_dict['names'])):
    df_pregunta = pd.read_csv('./../Datos_censo/Preguntas/'+pregs_dict['names'][i]+'.csv', encoding='latin-1')
    prefix = pregs_dict['prefix'][i]
    df_pregunta.columns = [prefix + unidecode.unidecode(col) for col in df_pregunta.columns]

#     edit_df_list = []
    df_pregunta.rename(columns = {df_pregunta.columns[0]: 'radio'}, inplace=True)
    df_pregunta = df_pregunta.iloc[:, :-1]

    df_pregunta['radio'] = df_pregunta['radio'].astype(str).str.zfill(9)

    df = pd.DataFrame([e['properties'] for e in data['features']])[['LINK', 'HOGARES', 'POBLACION_TOTAL', 'PROVINCIA']]

    df = df.merge(df_pregunta, left_on = 'LINK', right_on = 'radio', how = 'left').fillna(0).replace(0, 0.01)
    df_list += [df.set_index('radio')]
    
data_censo = pd.concat(df_list, axis = 1).T.drop_duplicates().T

In [28]:
data_censo.sample(10)

,LINK,HOGARES,POBLACION_TOTAL,PROVINCIA,Actividad 1 Ocupado,Actividad 2 Desocupado,Actividad 3 Inactivo,Grupo etario1 0 - 14,Grupo etario2 15 - 64,Grupo etario3 65 y mas,Sexo 1 Varon,Sexo 2 Mujer,Nacionalidad 1 Argentina,Nacionalidad 2 Otro pais,Sabe leer 1 Si,Sabe leer 2 No,Asist escuela 1 Asiste,Asist escuela 2 Asistio,Asist escuela 3 Nunca asistio,"Nivel educativo 1 Inicial (jardin, preescolar)",Nivel educativo 2 Primario,Nivel educativo 3 EGB,Nivel educativo 4 Secundario,Nivel educativo 5 Polimodal,Nivel educativo 6 Superior no universitario,Nivel educativo 7 Universitario,Nivel educativo 8 Post universitario,Nivel educativo 9 Educacion especial,Terminado 1 Si,Terminado 2 No,Urbana o rural 1 Urbano,Urbana o rural 2 Rural agrupado,Urbana o rural 3 Rural disperso,Tipo de vivienda particular 1 Casa,Tipo de vivienda particular 2 Rancho,Tipo de vivienda particular 3 Casilla,Tipo de vivienda particular 4 Departamento,Tipo de vivienda particular 5 Pieza en inquilinato,Tipo de vivienda particular 6 Pieza en hotel familiar o pension,Tipo de vivienda particular 7 Local no construido para habitacion,Tipo de vivienda particular 8 Vivienda movil,Tipo de vivienda particular 9 Persona/s viviendo en la calle
radio,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
065151216,065151216,300,1052,BUENOS AIRES,524,61,219,265,720,67,536,516,1018,34,929,65,283,676,35,36,410,73,326,41,25,47,0.01,1,414,508,254,0.01,0.01,236,1,6,9,1,0.01,1,0.01,0.01
780210901,780210901,513,1645,SANTA CRUZ,828,36,371,436,1099,110,862,783,1468,177,1453,87,494,1024,22,85,433,250,453,90,47,154,4,2,607,824,592,0.01,0.01,539,2,10,24,13,1,1,0.01,0.01
420210713,420210713,241,678,LA PAMPA,373,23,168,123,460,95,300,378,671,7,629,28,163,473,21,15,261,35,176,56,41,50,1,1,322,298,260,0.01,0.01,196,0.01,0.01,64,0.01,0.01,0.01,0.01,0.01
065150508,065150508,214,743,BUENOS AIRES,356,17,186,155,469,119,330,413,713,30,686,29,180,530,5,29,273,16,261,33,49,46,1,2,384,295,215,0.01,0.01,211,0.01,3,0.01,0.01,0.01,0.01,0.01,0.01
060561804,060561804,233,829,BUENOS AIRES,358,25,217,139,515,175,392,437,796,33,760,37,210,572,15,32,279,21,275,36,66,67,2,4,337,409,259,0.01,0.01,245,0.01,0.01,13,0.01,0.01,0.01,0.01,0.01
020121506,020121506,343,829,CIUDAD DE BUENOS AIRES,459,20,245,112,524,193,377,452,775,54,789,24,199,609,5,21,147,0.01,235,1,104,280,20,0.01,477,310,423,0.01,0.01,6,0.01,0.01,416,0.01,0.01,1,0.01,0.01
821330210,821330210,5,23,SANTA FE,9,1,6,8,14,1,13,10,23,0.01,21,2,10,11,2,1,13,0.01,6,1,0.01,0.01,0.01,0.01,6,14,0.01,0.01,9,7,1,1,0.01,0.01,0.01,0.01,0.01,0.01
500070605,500070605,189,504,MENDOZA,284,10,138,81,337,86,235,269,482,22,475,15,146,340,4,13,54,27,118,14,66,180,14,0.01,257,216,218,0.01,0.01,153,0.01,0.01,65,0.01,0.01,0.01,0.01,0.01
340140716,340140716,126,419,FORMOSA,171,5,117,137,242,40,229,190,389,30,358,38,156,213,27,10,226,19,86,7,13,7,1,0.01,116,243,0.01,205,0.01,166,36,3,0.01,0.01,0.01,0.01,0.01,0.01


### Maximum entropy

In [51]:
R = data_censo.loc['065151216'][[u'Nacionalidad 1 Argentina', u'Nacionalidad 2 Otro pais']]
C = data_censo.loc['065151216'][[u'Grupo etario1 0 - 14', u'Grupo etario2 15 - 64', u'Grupo etario3 65 y mas']]

In [48]:
# data_censo.plot(y = u'Actividad 1 Ocupado', x = u'Grupo etario2 15 - 64', marker = 'o', linewidth = 0, alpha = .002)

In [53]:
(C.sum() + R.sum())/2

1052.0

In [50]:
data_censo.loc['065151216']

LINK                                                                    065151216
HOGARES                                                                       300
POBLACION_TOTAL                                                              1052
PROVINCIA                                                            BUENOS AIRES
Actividad 1 Ocupado                                                           524
Actividad 2 Desocupado                                                         61
Actividad 3 Inactivo                                                          219
Grupo etario1 0 - 14                                                          265
Grupo etario2 15 - 64                                                         720
Grupo etario3 65 y mas                                                         67
Sexo 1 Varon                                                                  536
Sexo 2 Mujer                                                                  516
Nacionalidad 1 A

In [8]:
datos.groupby('CODUSU')['ingreso_usd'].describe().sample(20)

,count,mean,std,min,25%,50%,75%,max
CODUSU,,,,,,,,
TQRMNORRSHMNKTCDEIOAH00497270,12.0,109.543103,162.026109,-0.602121,0.000000,0.000000,327.821421,330.029458
TQRMNOQXUHMMRSCDEIJAH00496003,8.0,87.666999,193.430219,0.000000,0.000000,0.000000,38.468840,547.460630
TQRMNOQPPHMOKQCDEGPDJ00597014,16.0,417.021750,511.191834,0.000000,0.000000,0.000000,857.576411,1274.881091
TQRMNORXSHMMKPCDEIMBF00497008,4.0,139.429113,165.083920,0.000000,0.000000,117.079079,256.508192,323.558292
TQRMNOPXXHLNLSCDEFJAH00600667,2.0,1711.558444,488.830306,1365.903220,1538.730832,1711.558444,1884.386057,2057.213669
TQRMNOSPVHJMKSCDEGPDJ00476834,14.0,324.611269,393.898499,0.000000,0.000000,247.410264,462.046189,1097.198226
TQRMNOQRQHKMLQCDEIJAH00610390,11.0,34.633227,114.865418,0.000000,0.000000,0.000000,0.000000,380.965494
TQRMNOPXSHMMKUCDEGPDJ00597046,2.0,719.181725,212.210358,569.126342,644.154033,719.181725,794.209416,869.237108
TQRMNOQTUHMKKPCDEHMHF00511239,22.0,290.763056,404.278901,0.000000,0.000000,31.813616,377.155839,1272.544639


In [21]:
datos[['P47T','PONDII','V2_M','V3_M','V4_M','V5_M','V8_M','V9_M','V10_M','V11_M','V12_M','V18_M','V19_AM','V21_M','T_VI']].quantile(.95)

P47T      25000.0
PONDII     1856.0
V2_M       8000.0
V3_M          0.0
V4_M          0.0
V5_M          0.0
V8_M          0.0
V9_M          0.0
V10_M         0.0
V11_M         0.0
V12_M         0.0
V18_M         0.0
V19_AM        0.0
V21_M         0.0
T_VI       9900.0
Name: 0.95, dtype: float64

In [ ]:
sample_size = datos.groupby(['ANO4', 'TRIMESTRE']).count()['CODUSU'].mean()
total_pob = 4.4e7


In [ ]:
#Selecciona mayores de 18
ingreso = 'ingreso_usd'
datos_mayores = datos.loc[datos.CH06 >= 18]
# datos_mayores['P47T'] = datos_mayores[['P47T']].div(datos_mayores.groupby(['ANO4', 'TRIMESTRE'])[['P47T']].transform('median'))

datos_mayores['bin_ingreso'] = pd.cut(np.log10(datos_mayores[ingreso].clip(0.01, np.inf)), 40)
